In [2]:
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from PIL import Image

In [3]:
path_src = '/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images'
path_dst = '/kaggle/working/ODIR_TrainingDataSet_Preprocess'
image_width = 224

In [4]:
def remove_black_pixels(folder_src, folder_dst, file):
    """
    Remove all the necessary black pixels and crop the image accordingly
    """
    image = cv2.imread(join(folder_src, file))
    
    # Mask with colored pixels
    mask = image > 0

    # Pixel coordinates with color
    coordinates = np.argwhere(mask)

    # Define the rectangle with colored pixels
    x0, y0, s0 = coordinates.min(axis=0)
    x1, y1, s1 = coordinates.max(axis=0) + 1

    # Cutting the image according to the rectangle
    cropped = image[x0:x1, y0:y1]

    # Writing the image file to the destination directory
    cv2.imwrite(join(folder_dst, file), cropped)

In [5]:
def resize_image(folder_src, folder_dst, file, keep_ratio = False):
    """
       Redimension the image
    """
    img = Image.open(join(folder_src, file))

    # If ratio is True
    if keep_ratio:
        width_perc = image_width/float(img.size[0])
        height_size = int((float(img.size[1]) * float(width_perc)))
        img = img.resize((image_width, height_size), Image.LANCZOS)
    else:
        img = img.resize((image_width, image_width), Image.LANCZOS)

    # Switching the right/left image of the right eye
    if 'right' in file:
        img = img.transpose(Image.FLIP_LEFT_RIGHT)

    # Saving and closing the file
    img.save(join(folder_dst, file), optimize = True, quality = 100)
    img.close()

In [6]:
def equalize_image(folder_src, folder_dst, file):
    """
        Equalize the image histogram enhancing the contrast of the image
    """
    rgb_img = cv2.imread(join(folder_src,file))

    # Converting the image from RGB format to YcrCb format
    ycrcb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2YCrCb)

    # Equalization on channel Y
    ycrcb_img[:, :, 0] = cv2.equalizeHist(ycrcb_img[:, :, 0])

    # Converting image from YcrCb format to RBG format
    equalized_img = cv2.cvtColor(ycrcb_img, cv2.COLOR_YCrCb2BGR)
    cv2.imwrite(join(folder_dst,file), equalized_img)

In [ ]:
from tqdm import tqdm
if  not os.path.exists(PATH_DST):
    os.makedirs(PATH_DST)

files = [f for f in listdir(path_src) if isfile(join(path_src, f))]
for file in tqdm(files):
    # cropping
    remove_black_pixels(path_src, path_dst, file)
    # equalize
    equalize_image(path_dst,path_dst, file)
    # resize
    resize_image(path_dst,path_dst, file)

 44%|████▍     | 3075/7000 [19:35<31:02,  2.11it/s]  